In [1]:
import classes
import random
from amadeus import Client, ResponseError
from hotels1 import hotelFunctions
import amadeus
#import api
import hotels1
import hotelTests
import transport
from transport import transportFunctions
from transport import getNZfromCity
import datetime

In [2]:
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
hotel = hotelFunctions(amadeus)

In [3]:
hotels = hotel.getHotelsByGeocode(getNZfromCity('London')[0], getNZfromCity('London')[1])

Those are it: 'media'


In [4]:
for i in hotels:
    print(i)

{'Duration': None, 'Cost': 219.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'HOLIDAY INN EXP SOUTHWARK ST', 'Dest': '51.5061,-0.10144'}
{'Duration': None, 'Cost': 196.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Blackfriars', 'Dest': '51.5131,-0.10561'}
{'Duration': None, 'Cost': 207.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Waterloo', 'Dest': '51.50329,-0.11521'}
{'Duration': None, 'Cost': 367.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Andaz Liverpool Street London', 'Dest': '51.51651,-0.08191'}
{'Duration': None, 'Cost': 449.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'THE WALDORF HILTON', 'Dest': '51.51264,-0.11937'}
{'Duration': None, 'Cost': 285.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'DOUBLETREE TOWER OF LONDON', 'Dest': '51.51091,-0.07849'}
{'Duration': None, 'Cost': 202.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Tower Bridge', 'Dest': '51.50003,-0.0815'}
{'Duration': None, 'Cost': 221.63, 

In [5]:
transport = transportFunctions()
transports = transport.getTransport(51.5061,-0.10144,51.2191,-0.1246)


In [6]:
for i in transports:
    print(i)

{'Duration': datetime.timedelta(seconds=360), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 18, 2), datetime.datetime(2022, 6, 18, 18, 8)], 'Title': 'pedestrian', 'Dest': '51.508517,-0.103367'}
{'Duration': datetime.timedelta(seconds=360), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 18, 8), datetime.datetime(2022, 6, 18, 18, 14)], 'Title': 'pedestrian', 'Dest': '51.51181,-0.10333'}
{'Duration': datetime.timedelta(seconds=2280), 'Cost': None, 'Times:': [datetime.datetime(2022, 6, 18, 18, 14), datetime.datetime(2022, 6, 18, 18, 52)], 'Title': 'regionalTrain', 'Dest': '51.2402,-0.1659'}
{'Duration': datetime.timedelta(seconds=240), 'Cost': None, 'Times:': [datetime.datetime(2022, 6, 18, 19, 0), datetime.datetime(2022, 6, 18, 19, 4)], 'Title': 'regionalTrain', 'Dest': '51.22681,-0.13253'}
{'Duration': datetime.timedelta(seconds=60), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 19, 4), datetime.datetime(2022, 6, 18, 19, 5)], 'Title': 'pedestrian', 'Dest': '51.226568,

In [7]:
attractions = hotel.getActivities(32.079664, 34.767410, 2)

In [8]:
for i in attractions:
    print(i)

{'Duration': None, 'Cost': 25.0, 'Times:': [None, None], 'Title': 'Tel Aviv Pub Crawl', 'Dest': '32.075996,34.766022'}
{'Duration': None, 'Cost': 96.0, 'Times:': [None, None], 'Title': 'From Jerusalem: Masada and Dead Sea Tour', 'Dest': '32.075565,34.767686'}
{'Duration': None, 'Cost': 34.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Street Food Tour', 'Dest': '32.077840,34.777979'}
{'Duration': None, 'Cost': 80.0, 'Times:': [None, None], 'Title': 'Jerusalem Day Tour from Tel Aviv', 'Dest': '32.073954,34.765815'}
{'Duration': None, 'Cost': 65.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Jerusalem Old City Late Morning Tour', 'Dest': '32.074602,34.768443'}
{'Duration': None, 'Cost': 53.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Private Group Guided Bike Tour', 'Dest': '32.074815,34.767932'}


In [9]:
def chooseCheapestHotel(hotels):
    lowest = 1000000
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost < lowest):
            index = i
            lowest = hotels[i].cost
    return hotels[index]
def chooseMostLuxuriousHotel(hotels):
    highest = 0
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost > highest):
            index = i
            highest = hotels[i].cost
    return hotels[index]

In [53]:
lowCost = False
luxury = True
#shoomerShabat = False
tripPeople = 4
duration = 4
museumOriented = True
fastPaced = True
usedAttractions = []
maxPrice = 10000
priceCapLeft = 10000 # maybe add mostCentralLocation implementation later
amadeus = Client(
    client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
    client_secret='rKvILHDsjxcCh6yq'
)
city = "Tel Aviv"
hotel = hotelFunctions(amadeus)
transport = transportFunctions()
#print(city)
#print(getNZfromCity(city))
hotels = hotel.getHotelsByGeocode(getNZfromCity(city)[0], getNZfromCity(city)[1])
for i in hotels:
    print(i)
choosenHotel = None
if (lowCost):
    choosenHotel = chooseCheapestHotel(hotels)
elif (luxury):
    choosenHotel = chooseMostLuxuriousHotel(hotels)
else:
    choosenHotel = random.choice(hotels)
print(choosenHotel)

{'Duration': None, 'Cost': 153.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Best Western Regency Suites', 'Dest': '32.07979,34.76793'}
{'Duration': None, 'Cost': 467.1, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Sheraton Tel Aviv', 'Dest': '32.08171,34.76821'}
{'Duration': None, 'Cost': 467.1, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Sheraton Tel Aviv', 'Dest': '32.08171,34.76821'}


In [54]:
def cleanUsedAttractions(attractions, usedAttractions):
    newAttractionList = []
    for i in attractions:
        if (i.title.lower() in usedAttractions):
            continue
        newAttractionList.append(i)
    return newAttractionList

In [55]:
transport = transportFunctions()
def planFullDay(tripPeople, luxury, lowCost, museumOriented, 
                fastPaced, priceCapLeft, attractionObject, transportObject, hotel, usedAttractions, date):
    xCoord = hotel.destination.split(',')
    yCoord = float(xCoord[1])
    xCoord = float(xCoord[0])
    print(xCoord, yCoord)
    attractions = [] # maybe add most_central_location implementation later
    while (attractions == [] or len(attractions) < 3):
        attractions = attractionObject.getActivities(xCoord, yCoord, 10)
        attractions = cleanUsedAttractions(attractions, usedAttractions)
        noise = (random.random() - 0.5) * 0.2
        xCoord += noise
        yCoord += noise
    attractionOptions = []
    for i in attractions:
        print(i)
    if (museumOriented):
        for i in attractions:
            if ('museum' in i.title.lower()):
                attractionOptions.append(i)
        if (attractionOptions == []):
            attractionOptions = attractions
            noise = (random.random() - 0.5) * 0.2
            xCoord += noise
            yCoord += noise
    else:
        attractionOptions = attractions
    if (lowCost):
        attractionOptions.sort(key=lambda x: x.cost)
    elif (luxury):
        attractionOptions.sort(key=lambda x: x.cost, reverse=True)
    day, usedAttractions = planDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions)

In [56]:
date = datetime.datetime.now()
date = date.replace(minute=59, hour=23, second=59, year=2022, month=7, day=1)

In [59]:
usedAttractions = []
fastPaced = False
print(planFullDay(tripPeople, luxury, lowCost,
            museumOriented, fastPaced, priceCapLeft, hotel, transport, choosenHotel, usedAttractions, date))

32.08171 34.76821
{'Duration': None, 'Cost': 25.0, 'Times:': [None, None], 'Title': 'Tel Aviv Pub Crawl', 'Dest': '32.075996,34.766022'}
{'Duration': None, 'Cost': 96.0, 'Times:': [None, None], 'Title': 'From Jerusalem: Masada and Dead Sea Tour', 'Dest': '32.075565,34.767686'}
{'Duration': None, 'Cost': 34.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Street Food Tour', 'Dest': '32.077840,34.777979'}
{'Duration': None, 'Cost': 80.0, 'Times:': [None, None], 'Title': 'Jerusalem Day Tour from Tel Aviv', 'Dest': '32.073954,34.765815'}
{'Duration': None, 'Cost': 65.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Jerusalem Old City Late Morning Tour', 'Dest': '32.074602,34.768443'}
{'Duration': None, 'Cost': 53.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Private Group Guided Bike Tour', 'Dest': '32.074815,34.767932'}
THE DATE IS: 2022-07-01 09:00:00.750891
33.41528476239277 36.101784762392775 32.075565 34.767686 2022-07-01T09:00:00.750891
{'notices': [{'title': 'Routing is not possible du

IndexError: list index out of range

In [40]:
transport = transportFunctions()
#print(datetime.datetime.now())
date = datetime.datetime.now()
date = date.replace(minute=59, hour=23, second=59, year=2022, month=10, day=1)
a = transport.getTransportByTime(51.5061,-0.10144,51.2191,-0.1246, str(date.isoformat()))
for i in a:
    print(i)
    pass

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T06:50:00+01:00', 'place': {'type': 'place', 'location': {'lat': 51.5061, 'lng': -0.10144}}}, 'arrival': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S0-arr', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'arrival': {'time': '2022-10-02T07:09:00+01:00', 'place': {'name': 'London Bridge', 'type': 'station', 'location': {'lat': 51.50511, 'lng': -0.08609}, 'wheelchairAccessible': 'yes', 'id': '120011305', 'code': 'LNDNBDC'}}, 'spans': [{'walkAttributes': ['indoor']}], 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-10-02

In [44]:
def planDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions):
    currDate = date.replace(minute = 0, hour = 9, second=0)
    print("THE DATE IS:", currDate)
    attractionArr = []
    transportArr = []
    currX = xCoord
    currY = yCoord
    attractionCount = 2
    if (fastPaced):
        attractionCount = 3
    for i in range(0, min((attractionCount), len(attractionOptions))):
        attractionLocation = attractionOptions[i].destination.split(',')
        destX = float(attractionLocation[0])
        destY = float(attractionLocation[1])
        print(currX, currY, destX, destY, str(currDate.isoformat()))
        currTransport = transport.getTransportByTime(currX, currY, destX, destY, str(currDate.isoformat()))
        currX = destX
        currY = destY
        print(currTransport)
        currDate = currTransport[-1].timeEnd
        currDate = currDate + datetime.timedelta(minutes=5)
        transportArr.append(currTransport)
        attractionOptions[i].timeStart = currDate
        currDate = currDate + datetime.timedelta(hours=3)
        attractionOptions[i].timeEnd = currDate
        currDate = currDate + datetime.timedelta(minutes=60)
        attractionArr.append(attractionOptions[i])
        usedAttractions.append(attractionOptions[i].title.lower())
    print("The date at the end is", currDate)
    currTransport = transport.getTransportByTime(currX, currY, xCoord, yCoord, str(currDate.isoformat()))
    transportArr.append(currTransport)
    print("Found Attractions:\n\n\n")
    for i in attractionArr:
        print(i)
    print("Found Trip\n\n\n")
    for i in transportArr:
        for j in i:
            print(j)
    return [], []